In [23]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import analysis

In [2]:
orders_file = 'testcases/orders-01.csv'
orders_df = pd.read_csv(orders_file, index_col='Date', parse_dates=True, na_values=['nan'])

syms = orders_df['Symbol'].values
syms = np.unique(syms)

sd = orders_df.index[0]
ed = orders_df.index[-1]

print orders_df.head()

           Symbol Order  Shares
Date                           
2011-01-10   AAPL   BUY    1500
2011-01-13   AAPL  SELL    1500
2011-01-13    IBM   BUY    4000
2011-01-26   GOOG   BUY    1000
2011-02-02    XOM  SELL    4000 



In [3]:
def get_data(start, end, symbols):
    data = pd.DataFrame()
    for sym in symbols:
        data[sym] = web.DataReader(sym, 'yahoo', start, end)['Adj Close']
    data.columns = symbols
    return data

In [4]:
market_prices = get_data(sd, ed, syms)
market_prices['CASH'] = 1.0
market_rets = market_prices.pct_change().dropna()
print market_prices.head(), "\n"
print market_rets.head()

                 AAPL        GOOG         IBM        XOM  CASH
Date                                                          
2011-01-10  44.560027  306.798725  127.558085  63.269955   1.0
2011-01-11  44.454628  307.697825  127.247052  63.741558   1.0
2011-01-12  44.816367  308.127408  128.819503  64.491061   1.0
2011-01-13  44.980320  308.037501  128.577589  64.600537   1.0
2011-01-14  45.344657  311.778769  129.597080  65.552153   1.0 

                AAPL      GOOG       IBM       XOM  CASH
Date                                                    
2011-01-11 -0.002365  0.002931 -0.002438  0.007454   0.0
2011-01-12  0.008137  0.001396  0.012357  0.011758   0.0
2011-01-13  0.003658 -0.000292 -0.001878  0.001698   0.0
2011-01-14  0.008100  0.012145  0.007929  0.014731   0.0
2011-01-18 -0.022469  0.024752  0.004333  0.011177   0.0


In [14]:
trades_df = pd.DataFrame(data=0., columns=market_prices.columns, index=market_prices.index)

for index, row in orders_df.iterrows():
    if row['Order'] == 'SELL':
        trades_df[row['Symbol']].loc[index] = -1 * row['Shares']
        trades_df['CASH'].loc[index] = trades_df['CASH'].loc[index] + market_prices[row['Symbol']].loc[index] * row['Shares']
    else:
        trades_df[row['Symbol']].loc[index] = row['Shares']
        trades_df['CASH'].loc[index] = trades_df['CASH'].loc[index] - market_prices[row['Symbol']].loc[index] * row['Shares']
print trades_df.head(), "\n"
print trades_df.tail()

              AAPL  GOOG     IBM  XOM         CASH
Date                                              
2011-01-10  1500.0   0.0     0.0  0.0  -66840.0405
2011-01-11     0.0   0.0     0.0  0.0       0.0000
2011-01-12     0.0   0.0     0.0  0.0       0.0000
2011-01-13 -1500.0   0.0  4000.0  0.0 -446839.8760
2011-01-14     0.0   0.0     0.0  0.0       0.0000 

            AAPL    GOOG     IBM  XOM          CASH
Date                                               
2011-12-21  20.0     0.0     0.0  0.0   -1031.73154
2011-12-22   0.0     0.0     0.0  0.0       0.00000
2011-12-23   0.0     0.0     0.0  0.0       0.00000
2011-12-27   0.0  2200.0     0.0  0.0 -703572.61700
2011-12-28   0.0     0.0 -2200.0  0.0  355634.80040


In [22]:
#FINISH THIS PART

start_val = 1000000
shares_owned = pd.DataFrame(data=0., columns=market_prices.columns, index=market_prices.index)
shares_owned['CASH'][0] = start_val

for x in xrange(len(shares_owned)):
    if x > 0:
        shares_owned.iloc[x] = shares_owned.iloc[x-1]
    shares_owned.iloc[x] = shares_owned.iloc[x] + trades_df.iloc[x]

my_portfolio = shares_owned * market_prices
my_portfolio['Portfolio'] = my_portfolio.sum(axis=1)
print shares_owned.head(), "\n"
print shares_owned.tail(), "\n"
print my_portfolio.tail()

              AAPL  GOOG     IBM  XOM         CASH
Date                                              
2011-01-10  1500.0   0.0     0.0  0.0  933159.9595
2011-01-11  1500.0   0.0     0.0  0.0  933159.9595
2011-01-12  1500.0   0.0     0.0  0.0  933159.9595
2011-01-13     0.0   0.0  4000.0  0.0  486320.0835
2011-01-14     0.0   0.0  4000.0  0.0  486320.0835 

            AAPL    GOOG     IBM  XOM          CASH
Date                                               
2011-12-21  20.0 -1255.0  2200.0  0.0  1.411472e+06
2011-12-22  20.0 -1255.0  2200.0  0.0  1.411472e+06
2011-12-23  20.0 -1255.0  2200.0  0.0  1.411472e+06
2011-12-27  20.0   945.0  2200.0  0.0  7.078993e+05
2011-12-28  20.0   945.0     0.0  0.0  1.063534e+06 

                  AAPL           GOOG          IBM  XOM          CASH  \
Date                                                                    
2011-12-21  1031.73154 -392310.412190  350763.8772  0.0  1.411472e+06   
2011-12-22  1037.19662 -394742.702590  351865.6174  0.0 

In [28]:
# compute stats... need to update analysis.py
spy = analysis.get_data(sd, ed, ['SPY'])
spy = analysis.allocate_data(spy, [1.0], start_val)
cr, adr, sddr, sr = analysis.compute_portfolio_stats(my_portfolio)
ev = my_portfolio['Portfolio'][-1]

print "Start Date:", sd
print "End Date:", ed
print "Symbols:", syms
print "Sharpe Ratio: %5.3f" % sr
print "Volatility (stdev of daily returns): %5.3f" % sddr
print "Average Daily Return: %7.5f" % adr
print "Cumulative Return: %5.3f" % cr
print "Final Value: %10.2f" % ev


Start Date: 2011-01-10 00:00:00
End Date: 2011-12-28 00:00:00
Symbols: ['AAPL' 'GOOG' 'IBM' 'XOM']
Sharpe Ratio: 1.132
Volatility (stdev of daily returns): 0.020
Average Daily Return: 0.00145
Cumulative Return: 0.367
Final Value: 1366538.81
